In [75]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [76]:
from dotenv import load_dotenv
import os
from openai import OpenAI

In [77]:
# Load environment variables from .env file
load_dotenv()

# Get OpenAI API key from environment variables
openai_api_key = os.getenv('OPENAI_API_KEY')

In [78]:
def build_prompt(docs, question):
    context = ""

    for doc in docs:
        context = context + f"section: {doc['section']}\n"
        context = context + f"question: {doc['question']}\n"
        context = context + f"answer: {doc['text']}\n\n"
    
    prompt_template = """
    You are a course teaching assistant. Answer the QUESTION based on the CONTEXT. 
    Use only the facts from the CONTEXT. If you don't know the answer, say that you don't know.
    If the question is not related to the CONTEXT, say that you don't know.
    If the question is not clear, ask for more details.

    QUESTION: {question}

    CONTEXT: {context}
    """

    prompt = prompt_template.format(question=question, context=context).strip()

    return prompt

In [79]:
def llm(prompt):
    client = OpenAI(api_key=openai_api_key) 

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}]
    )

    return response.choices[0].message.content

In [80]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(search_results, query)
    answer = llm(prompt)
    return answer

## Vector Search

In [81]:
from qdrant_client import QdrantClient, models

qd_client = QdrantClient("http://localhost:6333")

In [82]:
EMBEDDING_DIMENSIONALITY = 512
model_handle = "jinaai/jina-embeddings-v2-small-en"

In [83]:
collection_name = "zoomcamp-faq"

In [84]:
qd_client.delete_collection(collection_name=collection_name)

True

In [85]:
# Create the collection with specified vector parameters
qd_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY, 
        distance=models.Distance.COSINE
    )
)

True

In [86]:
qd_client.create_payload_index(
    collection_name=collection_name,
    field_name="course",
    field_schema="keyword" # exact matching on string metadata fields
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

In [87]:
points = []

for i, doc in enumerate(documents):
    text = doc['question'] + ' ' + doc['text']
    vector = models.Document(text=text, model=model_handle)
    point = models.PointStruct(
        id=i,
        vector=vector,
        payload=doc
    )
    points.append(point)

In [88]:
points[0]

PointStruct(id=0, vector=Document(text="Course - When will the course start? The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.", model='jinaai/jina-embeddings-v2-small-en', options=None), payload={'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with ann

In [89]:
qd_client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

In [90]:
question = "I just discovered the course. Can I still join it?"

In [91]:
def vector_search(question):
    print("Vector search is used")
    course = "data-engineering-zoomcamp"
    query_points = qd_client.query_points(
        collection_name=collection_name,
        query=models.Document(
            text=question,
            model=model_handle 
        ),
        query_filter=models.Filter( # filter by course name
                must=[  
                    models.FieldCondition(
                        key="course",
                        match=models.MatchValue(value=course)
                    )
                ]
            ),
        limit=5,
        with_payload=True
    )

    results = []

    for point in query_points.points:
        results.append(point.payload)

    return results

In [92]:
vector_search('I just discovered the course. Can I still join it?')

Vector search is used


[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [93]:
def rag(query):
    search_results = vector_search(query)
    prompt = build_prompt(search_results, query)
    answer = llm(prompt)
    return answer

In [94]:
rag('I just discovered the course. Can I still join it?')

Vector search is used


"Yes, you can still join the course after the start date. Even if you don't register, you're still eligible to submit the homeworks. Just be mindful of the deadlines for turning in the final projects."